LDA on Google Collab


In [0]:
!pip install -q findspark
!pip install pyspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!tar xf spark-2.4.0-bin-hadoop2.7.tgz

    100% |████████████████████████████████| 213.4MB 90kB/s 
    100% |████████████████████████████████| 204kB 27.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/cd/54/c2/abfcc942eddeaa7101228ebd6127a30dbdf903c72db4235b23
Successfully built pyspark


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"

In [0]:
import pyspark as ps
import warnings
from pyspark.sql import SQLContext
  
sc = ps.SparkContext('local[4]')
sqlContext = SQLContext(sc)

In [0]:
 !unzip cookbook_text.zip

Archive:  cookbook_text.zip
   creating: cookbook_text/
  inflating: cookbook_text/amem.txt  
  inflating: cookbook_text/amwh.txt  
  inflating: cookbook_text/army.txt  
  inflating: cookbook_text/aunt.txt  
  inflating: cookbook_text/bart.txt  
  inflating: cookbook_text/beec.txt  
  inflating: cookbook_text/blue.txt  
  inflating: cookbook_text/bost.txt  
  inflating: cookbook_text/brkf.txt  
  inflating: cookbook_text/buck.txt  
  inflating: cookbook_text/cclu.txt  
  inflating: cookbook_text/chas.txt  
  inflating: cookbook_text/chin.txt  
  inflating: cookbook_text/choc.txt  
  inflating: cookbook_text/comm.txt  
  inflating: cookbook_text/conf.txt  
  inflating: cookbook_text/coow.txt  
  inflating: cookbook_text/creo.txt  
  inflating: cookbook_text/dcvb.txt  
  inflating: cookbook_text/dish.txt  
  inflating: cookbook_text/dome.txt  
  inflating: cookbook_text/econ.txt  
  inflating: cookbook_text/ency.txt  
  inflating: cookbook_text/engl.txt  
  inflating: cookbook_text/epia.

In [0]:
from pyspark.sql import SQLContext, Row
#from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from pyspark.ml.feature import CountVectorizer
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vector, Vectors

#for path in cookbook_text:
path = "cookbook_text"

data = sc.textFile(path).zipWithIndex().map(lambda wordsID: Row(word_id = wordsID[1], words = wordsID[0].split(" ")))
docDF = sqlContext.createDataFrame(data)
Vector1 = CountVectorizer(inputCol="words", outputCol="vectors",)
model = Vector1.fit(docDF)
result = model.transform(docDF)

corpus = result.select("word_id", "vectors").rdd.map(lambda z: [z[0],Vectors.fromML(z[1])]).cache()


In [0]:
# Cluster the documents into five distinct topics
#Used LDA Model to classify text using default conditions
lda_model = LDA.train(corpus, k=5,maxIterations=100,optimizer='online')
topics = lda_model.topicsMatrix()
vocabArray = model.vocabulary


# number of words per topic
wordNumbers = 15  
topicIndices = sc.parallelize(lda_model.describeTopics(maxTermsPerTopic = wordNumbers))

In [0]:
# specifing vector id of words to the actual words in the file

def render(selected_topic):  
    terms = selected_topic[0]
    result = []
    for i in range(wordNumbers):
        term = vocabArray[terms[i]]
        result.append(term)
    return result

#calling the render function
final_topics = topicIndices.map(lambda topic: render(topic)).collect()  

In [0]:
for topics in range(len(final_topics)):
    print ("Topic" + str(topics) + ":")
    for phrase in final_topics[topics]:
        print (phrase)
    print ('\n')

Topic0:
M.
E.
A.
of
B.
S.
C.
J.
H.
Red
and
D.
W.
L.
CHAPTER


Topic1:

&#160;
-
&#224;
la
--
de
"
OF
AND
Cream
No.
of,
THE
au


Topic2:

the
and
of
a
in
with
it
to
or
.
one
them
is
into


Topic3:
the
of
and
to
is
in
a
be
as
are
that
for
or
it
by


Topic4:
de

MRS.
SAUCE
TO
OR
AND
WITH
et
en
FOR
CHICKEN
la
DE
au


